# Fermi-Hubbard Model II

In [ ]:
using Pkg; Pkg.activate()

using KadanoffBaym, FFTW, Interpolations
using LinearAlgebra, BlockArrays

using JLD

In [ ]:
using PyPlot
PyPlot.plt.style.use("./paper.mplstyle")
using LaTeXStrings

## Model

### Hamiltonian

$$
\begin{align}\begin{split}
    \hat{H} &= - J \sum_{\langle{i,\,j}\rangle}\sum_\sigma \hat{c}^{\dagger}_{i,\sigma} \hat{c}^{\phantom{\dagger}}_{i+1,\sigma} + U\sum_{i=1}^L  \hat{c}^{\dagger}_{i,\uparrow} \hat{c}^{\phantom{\dagger}}_{i,\uparrow}   \hat{c}^{\dagger}_{i,\downarrow} \hat{c}^{\phantom{\dagger}}_{i,\downarrow}, 
\end{split}\end{align}
$$

### Green functions

$$
    G^>_{\uparrow,ij}(t, t') = -i \left\langle \hat{c}^{\phantom{\dagger}}_{i,\uparrow}(t) \hat{c}^{{\dagger}}_{i,\uparrow}(t') \right\rangle\\
    G^>_{\downarrow,ij}(t, t') = -i \left\langle \hat{c}^{\phantom{\dagger}}_{i,\downarrow}(t) \hat{c}^{{\dagger}}_{i,\downarrow}(t') \right\rangle\\
$$

### Self-energies

Hartree-Fock:
$$
    \Sigma^{\mathrm{HF}}_{\uparrow,\,ij}(t, t') = {\mathrm{i}}\delta_{ij}\delta(t - t') G^<_{\downarrow,ii}(t, t)\\
    \Sigma^{\mathrm{HF}}_{\downarrow,\,ij}(t, t') = {\mathrm{i}}\delta_{ij}\delta(t - t') G^<_{\uparrow,ii}(t, t)
$$


Second-order Born approximation:
$$
    \Sigma_{ij, \uparrow}  (t, t') = U^2 G_{ij, \uparrow}(t, t') G_{ij, \downarrow}(t, t') G_{ji, \downarrow}(t', t),\\
    \Sigma_{ij, \downarrow}(t, t') = U^2 G_{ij, \downarrow}(t, t') G_{ij, \uparrow}(t, t') G_{ji, \uparrow}(t', t)
$$


$T$-matrix approximation:
$$
    \Sigma_{ij, \uparrow}  (t, t') = i U^2 T_{ij}(t, t') G_{ji, \downarrow}(t', t),\\
    \Sigma_{ij, \downarrow}(t, t') = i U^2 T_{ij}(t, t') G_{ji, \uparrow}(t', t)
$$

$$
   \boldsymbol{T}(t, t') =  \boldsymbol{\Phi}(t, t') - U \int_{\mathcal{C}}\mathrm{d}s\; \boldsymbol{\Phi}(t, s) \boldsymbol{T}(s, t')
$$

$$
    \Phi_{ij}(t, t') = -i G_{ij, \uparrow}(t, t') G_{ij, \downarrow}(t, t')
$$

$$
   \boldsymbol{T}^\lessgtr(t, t') =  \boldsymbol{\Phi}^\lessgtr(t, t') - U \int_{0}^{t}\mathrm{d}s\; [\boldsymbol{\Phi}^>(t, s) - \boldsymbol{\Phi}^<(t, s)] \boldsymbol{T}^\lessgtr(s, t') - U \int_{0}^{t'}\mathrm{d}s\; \boldsymbol{\Phi}^\lessgtr(t, s)[\boldsymbol{T}^<(s, t') - \boldsymbol{T}^>(s, t')]
$$

## Solving

In [ ]:
function integrate1(hs::Vector, t1, t2, A::GreenFunction, B::GreenFunction, C::GreenFunction; tmax=t1)
    retval = zero(A[t1,t1])

    @inbounds for k in 1:tmax
        @views LinearAlgebra.mul!(retval, A[t1, k] - B[t1, k], C[k, t2], hs[k], 1.0)
    end
    return retval
end

function integrate2(hs::Vector, t1, t2, A::GreenFunction, B::GreenFunction, C::GreenFunction; tmax=t2)
    retval = zero(A[t1,t1])

    @inbounds for k in 1:tmax
        @views LinearAlgebra.mul!(retval, A[t1, k], B[k, t2] - C[k, t2], hs[k], 1.0)
    end
    return retval
end

In [ ]:
function fixed_point(F::Function, x0::AbstractArray; 
        mixing::Float64=0.5, 
        abstol::Float64=1e-12, 
        maxiter::Int=1000, 
        verbose::Bool=true, 
        norm=x -> LinearAlgebra.norm(x, Inf)
    )
    
    x_old = copy(x0)

    step = 0
    while step < maxiter
        x = F(x_old)
        res = norm(x - x_old)
        if verbose
            @info "step: $step // res: $res"
        end
        if res < abstol
            break
        end
        @. x_old = mixing * x + (1.0 - mixing) * x_old
        step += 1
    end

    if step == maxiter
        @warn "No convergence reached."
    end

    return x_old
end

In [ ]:
# Lattice size
L = 8

# Allocate the initial Green functions (time arguments at the end)
GL_u = GreenFunction(zeros(ComplexF64, L, L, 1, 1), SkewHermitian)
GG_u = GreenFunction(zeros(ComplexF64, L, L, 1, 1), SkewHermitian)
GL_d = GreenFunction(zeros(ComplexF64, L, L, 1, 1), SkewHermitian)
GG_d = GreenFunction(zeros(ComplexF64, L, L, 1, 1), SkewHermitian);

In [ ]:
# Initial conditions
N_u = zeros(L)
N_d = zeros(L)

N_u[1:4] = 0.1 .* [1, 1, 1, 1]
N_d[1:4] = 0.1 .* [1, 1, 1, 1]

N_u[5:8] = 0.0 .* [1, 1, 1, 1]
N_d[5:8] = 0.0 .* [1, 1, 1, 1]

GL_u[1, 1] = 1.0im * diagm(N_u)
GG_u[1, 1] = -1.0im * (I - diagm(N_u))
GL_d[1, 1] = 1.0im * diagm(N_d)
GG_d[1, 1] = -1.0im * (I - diagm(N_d));

In [ ]:
Base.@kwdef struct FermiHubbardDataTM{T}
    GL_u::T
    GG_u::T
    GL_d::T
    GG_d::T

    ΣL_u::T = zero(GL_u)
    ΣG_u::T = zero(GG_u)
    ΣL_d::T = zero(GL_d)
    ΣG_d::T = zero(GG_d)

    TL::T = zero(GL_u)
    TG::T = zero(GG_u)

    ΦL::T = zero(GL_u)
    ΦG::T = zero(GG_u)
end

data = FermiHubbardDataTM(GL_u=GL_u, GG_u=GG_u, GL_d=GL_d, GG_d=GG_d)

# Initialize T-matrix
data.TL[1, 1] = -1.0im .* GL_u[1, 1] .* GL_d[1, 1]
data.TG[1, 1] = -1.0im .* GG_u[1, 1] .* GG_d[1, 1]
data.ΦL[1, 1] = data.TL[1, 1]
data.ΦG[1, 1] = data.TG[1, 1];

In [ ]:
Base.@kwdef struct FermiHubbardModel{T}
    # interaction strength
    U::T

    # 8-site 3D cubic lattice
    h = begin
        h = BlockArray{ComplexF64}(undef_blocks, [4, 4], [4, 4])
        diag_block = [0 -1 0 -1; -1 0 -1 0; 0 -1 0 -1; -1 0 -1 0]
        setblock!(h, diag_block, 1, 1)
        setblock!(h, diag_block, 2, 2)
        setblock!(h, Diagonal(-1 .* ones(4)), 1, 2)
        setblock!(h, Diagonal(-1 .* ones(4)), 2, 1)

        h |> Array
    end

    H_u = h
    H_d = h
end

# Interaction parameter
const U₀ = 2.0
model = FermiHubbardModel(U = t -> U₀);

In [ ]:
# Right-hand side for the "vertical" evolution
function fv!(model, data, out, times, h1, h2, t, t′)
    # Unpack data and model
    (; GL_u, GG_u, GL_u, GG_d, ΣL_u, ΣG_u, ΣL_d, ΣG_d) = data
    (; H_u, H_d, U) = model

    # Real-time collision integrals
    ∫dt1(A, B, C) = integrate1(h1, t, t′, A, B, C)
    ∫dt2(A, B, C) = integrate2(h2, t, t′, A, B, C)
    
    # The interaction varies as a function of the forward time (t+t')/2
    U_t = U((times[t] + times[t′])/2)
    
    # Hartree-Fock self-energies
    ΣHF_u(t, t′) = im * U_t * Diagonal(GL_d[t, t])
    ΣHF_d(t, t′) = im * U_t * Diagonal(GL_u[t, t])
    
    # Equations of motion
    out[1] = -1.0im * ((H_u + ΣHF_u(t, t′)) * GL_u[t, t′] + 
            ∫dt1(ΣG_u, ΣL_u, GL_u) + ∫dt2(ΣL_u, GL_u, GG_u)
        )

    out[2] = -1.0im * ((H_u + ΣHF_u(t, t′)) * GG_u[t, t′] + 
            ∫dt1(ΣG_u, ΣL_u, GG_u) + ∫dt2(ΣG_u, GL_u, GG_u)
        )

    out[3] = -1.0im * ((H_d + ΣHF_d(t, t′)) * GL_d[t, t′] + 
            ∫dt1(ΣG_d, ΣL_d, GL_d) + ∫dt2(ΣL_d, GL_d, GG_d)
        )

    out[4] = -1.0im * ((H_d + ΣHF_d(t, t′)) * GG_d[t, t′] +
            ∫dt1(ΣG_d, ΣL_d, GG_d) + ∫dt2(ΣG_d, GL_d, GG_d)
        )  
    
    return out
end

function fd!(model, data, out, times, h1, h2, t, t′)
    fv!(model, data, out, times, h1, h2, t, t)
    out .-= adjoint.(out)
end

## T-matrix

In [ ]:
# Callback function for the self-energies
function T_matrix!(model, data, times, h1, h2, t, t′)
    # Unpack data and model
    (; GL_u, GG_u, GL_d, GG_d, TL, TG, ΣL_u, ΣG_u, ΣL_d, ΣG_d, ΦL, ΦG) = data
    (; U) = model
        
    # Real-time collision integral
    ∫dt1(A, B, C) = integrate1(h1, t, t′, A, B, C)
    ∫dt2(A, B, C) = integrate2(h2, t, t′, A, B, C)
    
    # Resize self-energies etc. when Green functions are resized
    if (n = size(GL_u, 3)) > size(ΣL_u, 3)
        resize!(ΣL_u, n)
        resize!(ΣG_u, n)
        resize!(ΣL_d, n)
        resize!(ΣG_d, n)
        
        resize!(TL, n)
        resize!(TG, n)
        resize!(ΦL, n)
        resize!(ΦG, n)
    end
    
    # The interaction varies as a function of the forward time (t+t')/2
    U_t = U((times[t] + times[t′])/2)
    
    # Set all Φs at the very first t′ since they are all known by then
    if t′ == 1
        for t′ in 1:t
            ΦL[t, t′] = -1.0im .* GL_u[t, t′] .* GL_d[t, t′]
            ΦG[t, t′] = -1.0im .* GG_u[t, t′] .* GG_d[t, t′]
        end
    end   
    
    # Solve VIEs implicitly (uncached version)
    TL[t, t′], TG[t, t′] = fixed_point([ΦL[t, t′], ΦG[t, t′]]; mixing=0.5, verbose=false) do x
        TL[t, t′], TG[t, t′] = x[1], x[2]
        
        [
            ΦL[t, t′] - U_t * (∫dt1(ΦG, ΦL, TL) + ∫dt2(ΦL, TL, TG)),
            ΦG[t, t′] - U_t * (∫dt1(ΦG, ΦL, TG) + ∫dt2(ΦG, TL, TG))
        ]
    end
    
#     # Solve VIEs implicitly (cached version)
#     TL[t, t′], TG[t, t′] = let
#         ∫dt_(x...) = (x[1] < 1 || x[2] < 1) ? zero(x[3][1,1]) : ∫dt(x...)
        
#         ΦL_ = ΦL[t, t′]
#         ΦG_ = ΦG[t, t′]
        
#         I1 = integrate1(h1, t, t′, ΦG, ΦL, TL; tmax=t-1)
#         I2 = integrate2(h2, t, t′, ΦL, TL, TG; tmax=t′-1)
#         I3 = integrate1(h1, t, t′, ΦG, ΦL, TG; tmax=t-1)
#         I4 = integrate2(h2, t, t′, ΦG, TL, TG; tmax=t′-1)
        
#         L_ = ΦL[t, t′] - U_t * (I1 + I2)
#         G_ = ΦG[t, t′] - U_t * (I3 + I4)
        
#         fixed_point([L_, G_]; mixing=0.5, verbose=false) do x
#             TL[t, t′], TG[t, t′] = x[1], x[2]

#             [
#                 L_ - U_t * (h1[t] * (ΦG[t,t] - ΦL[t,t]) * TL[t,t′] + h2[t′] * ΦL[t,t′] * (TL[t′,t′] - TG[t′,t′])),
#                 G_ - U_t * (h1[t] * (ΦG[t,t] - ΦL[t,t]) * TG[t,t′] + h2[t′] * ΦG[t,t′] * (TL[t′,t′] - TG[t′,t′]))
#             ]
#         end
#     end           
    
    # Define the self-energies
    ΣL_u[t, t′] = 1.0im .* U_t^2 .* TL[t, t′] .* transpose(GG_d[t′, t])
    ΣL_d[t, t′] = 1.0im .* U_t^2 .* TL[t, t′] .* transpose(GG_u[t′, t])
    
    ΣG_u[t, t′] = 1.0im .* U_t^2 .* TG[t, t′] .* transpose(GL_d[t′, t])
    ΣG_d[t, t′] = 1.0im .* U_t^2 .* TG[t, t′] .* transpose(GL_u[t′, t])
end

In [ ]:
tmax = 8 # 32;
tols = [(1e-8, 1e-10), (1e-6, 1e-8), (1e-4, 1e-6)];

In [ ]:
for (rtol, atol) in tols
    @time sol = kbsolve!(
        (x...) -> fv!(model, data, x...),
        (x...) -> fd!(model, data, x...),
        [data.GL_u, data.GG_u, data.GL_d, data.GG_d],
        (0.0, tmax);
        callback = (x...) -> T_matrix!(model, data, x...),
        atol = atol,
        rtol = rtol,
        dtini=1e-10,
        stop = x -> (println("t: $(x[end])"); flush(stdout); false)
    );

    save("FH_3D_T_matrix_sol_U_$(U₀)_tmax_$(tmax)_atol_$(atol)_rtol_$(rtol).jld", "solution", sol)
end

In [ ]:
# Callback function for the self-energies
function second_Born!(model, data, times, _, _, t, t′)
    # Unpack data and model
    (; GL_u, GG_u, GL_d, GG_d, ΣL_u, ΣG_u, ΣL_d, ΣG_d) = data
    (; U) = model
        
    # Resize self-energies when Green functions are resized    
    if (n = size(GL_u, 3)) > size(ΣL_u, 3)
        resize!(ΣL_u, n)
        resize!(ΣG_u, n)
        resize!(ΣL_d, n)
        resize!(ΣG_d, n)        
    end
    
    # The interaction varies as a function of the forward time (t+t')/2
    U_t = U((times[t] + times[t′])/2)
    
    # Define the self-energies
    ΣL_u[t, t′] = U_t^2 .* GL_u[t, t′] .* GL_d[t, t′] .* transpose(GG_d[t′, t])
    ΣL_d[t, t′] = U_t^2 .* GL_u[t, t′] .* GL_d[t, t′] .* transpose(GG_u[t′, t])
    
    ΣG_u[t, t′] = U_t^2 .* GG_u[t, t′] .* GG_d[t, t′] .* transpose(GL_d[t′, t])
    ΣG_d[t, t′] = U_t^2 .* GG_u[t, t′] .* GG_d[t, t′] .* transpose(GL_u[t′, t])
end

# Solve second Born for comparison
@time sol_2B = kbsolve!(
    (x...) -> fv!(model, data, x...),
    (x...) -> fd!(model, data, x...),
    [data.GL_u, data.GG_u, data.GL_d, data.GG_d],
    (0.0, tmax);
    callback = (x...) -> second_Born!(model, data, x...),
    atol = 1e-8,
    rtol = 1e-6,
    dtini=1e-10,
    stop = x -> (println("t: $(x[end])"); flush(stdout); false)
);

## Example plots

In [ ]:
loaded_sol = []
GFs = []
ts = []

for (i, (rtol, atol)) in enumerate(tols)
    sol = load("FH_3D_T_matrix_sol_U_$(U₀)_tmax_$(tmax)_atol_$(atol)_rtol_$(rtol).jld")
    push!(loaded_sol, sol)
    push!(GFs, sol["solution"].u)
    push!(ts, sol["solution"].t)
end;

In [ ]:
xpad = 8
ypad = 5;

figure(figsize = (8, 3))

ax = subplot(121)
idx_1 = 1
ms = 4

colors = ["C2", "C0", "C3"]
styles = ["s", "D", "o"]
ls = ["-", "-.", "--"]

plot([], [], label="\$\\texttt{rtol}\$", c="w")
for (i, (rtol, atol)) in enumerate(tols)
    plot(ts[i], [imag(GFs[i][1][idx_1, idx_1, k, k] + GFs[i][3][idx_1, idx_1, k, k]) for k in eachindex(ts[i])], 
        label = L"\texttt{%$(rtol)}", lw=1.5, styles[i], ms=ms, c = colors[i])
end

plot(sol_2B.t, [imag(sol_2B.u[1][idx_1, idx_1, k, k] .+ sol_2B.u[3][idx_1, idx_1, k, k]) for k in eachindex(sol_2B.t)], 
    lw=3, ls = "--", c = "k", alpha=0.75)

xlim(0, tmax)
ax.set_xticks(0:tmax / 4:tmax)
ylim(0, 0.3)
xlabel("\$J t\$")
ylabel("Charge on site 1", labelpad = 8)
ax.xaxis.set_tick_params(pad = xpad)
ax.yaxis.set_tick_params(pad = ypad)
ticklabel_format(axis = "y", style = "sci", scilimits = (-0, 0))
ax.legend(loc = "best", handlelength = 1, frameon = false, borderpad = 0, labelspacing = 0.25, ncol=2)

ax = subplot(122)
for (i, (rtol, atol)) in enumerate(reverse(tols))
    semilogy(ts[i], [tr(GFs[i][1][k, k]) + tr(GFs[i][3][k, k]) |> imag for k in eachindex(ts[i])] .- sum(N_u .+ N_d) .|> abs, 
        ls = ls[i], c = colors[i])
end

xlim(0, tmax)
ylim(1e-10, 2e-4)
ax.set_xticks(0:tmax / 4:tmax)
ylabel("\$Q(t) - Q_0\$", labelpad = 16)
xlabel("\$J t\$")
ax.xaxis.set_tick_params(pad = xpad)
ax.yaxis.set_tick_params(pad = ypad)
# ticklabel_format(axis = "y", style = "sci", scilimits = (-0, 0))
ax.yaxis.set_label_position("right")

tight_layout(pad = 0.1, w_pad = 0.5, h_pad = 0)
# savefig("fermi_hubbard_T_matrix_convergence.pdf")